# Execução prática de uma prova de conceito
## Execução prática de uma prova de conceito utilizando pydantic de FastAPI
Meu intuito é simular a integração de dois sistemas da area médica, trocando informações através de um protocolo, onde um dos sistemas faça a validação dos dados do protocolo


In [1]:
# instalando as bibliotecas necessárias para a POC

!pip install pydantic
!pip install pydantic[email]
!pip install FastAPI

In [6]:
# importando as bibliotecas necessárias para a execução da POC

from datetime import datetime
from typing import Optional
from uuid import uuid4
from enum import auto, IntFlag

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

In [17]:
# criando o objeto protocolo que será utilizado para integrar os sistemas da area médica

# classe responsavel por criar o objeto do tipo de arquivo a ser transportado pelo protocolo
class TipoArquivo(IntFlag):
  Texto = auto()
  Imagem = auto()
  Video = auto()


# classe responsavel por criar o objeto protocolo com os dados a serem transportados pelos sistemas
class Protocolo(BaseModel):

    # herança do basemodel para travar o objeto e não aceitar campos extras
    model_config = {
        "extra": "forbid",
    }


    __protocolos__ = []

    codigo: UUID4 = Field(default_factory=uuid4, description="Chave primária do protocolo", kw_only=True)

    sistema_origem: str = Field(description="Nome do sistema de origem", examples=["Cerner, PACS, VNA"])

    email_analista_responsavel: EmailStr = Field(description="Email do analista responsável pelo sistema", examples=["analista@cerner.com"], frozen=True)

    data_protocolo: Optional[datetime] = Field(default_factory=datetime.now, description="Data do envio do protocolo", kw_only=True)

    tipo: TipoArquivo = Field(default=None, description="Enumerador do tipo do protocolo a ser enviado: Texto, Imagem ou Video", kw_only=True)

    endereco_arquivo: str = Field(description="Localização do arquivo", examples=["www.arquivo.com, ftp"])


    @field_serializer("codigo", when_used="json")
    def serialize_id(self, codigo: UUID4) -> str:
        '''
        responsavel por serializar a chave primária para JSON
        :param id:
        :return:
        '''
        return str(codigo)

In [79]:
# endpoints disponibilizados por um dos sistemas para enviar e receber dados de protocolos de comunicação

# instanciando a api que simula um dos sistemas de comunicação
app = FastAPI()


@app.get("/protocolo", response_model=list[Protocolo])
async def getProtocolo() -> list[Protocolo]:
    print("retornando lista de protocolos")
    return list(Protocolo.__protocolos__)


@app.post("/protocolo", response_model=Protocolo)
async def createProtocolo(protocolo: Protocolo):
    print("Recebendo protocolo")
    Protocolo.__protocolos__.append(protocolo)
    return protocolo


@app.get("/protocolo/{codigo}", response_model=Protocolo)
async def getProtocolo(codigo: UUID4) -> Protocolo | JSONResponse:
    try:
        print("realizando busca de protocolo por código")
        # busca de protocolo pelo código
        return next((protocolo for protocolo in Protocolo.__protocolos__ if protocolo.codigo == codigo))
    except StopIteration:
        erro = f"Protocolo de código {codigo} não encontrado"
        return JSONResponse(status_code=404, content={"mensagem": erro})

In [88]:
# definindo uma função principal que irá simular a integração entre dois sistemas

def enviaProtocolo(verbose = False) -> None:


    with TestClient(app) as client:

        # enviando o primeiro protocolo
        response = client.post(
            "/protocolo",
            json={
                "sistema_origem": "Cerner",
                "email_analista_responsavel": "fulano@beltrano.com.br",
                "data_protocolo": "2025-03-12",
                "tipo": TipoArquivo.Texto,
                "endereco_arquivo": "www.arquivo.com/teste.pdf",
            },
        )
        if verbose: print(response.json())
        assert response.status_code == 200
        assert response.json()["sistema_origem"] == "Cerner", "O nome do sistema de origem do protocolo deveria ser Cerner, Pacs, VNA"
        assert response.json()["codigo"], "O protocolo deve ter um código"

        # fazendo o parse da resposta json para o objeto
        protocolo = Protocolo.model_validate(response.json())
        assert str(protocolo.codigo) == response.json()["codigo"], "O código deve ser o mesmo"
        assert protocolo.data_protocolo, "A data do protocolo deve ser informada"
        assert protocolo.tipo, "O tipo do arquivo do protocolo deve ser informado"
        assert protocolo.endereco_arquivo, "A localização do arquivo deve ser informado"



        # enviando o segundo protocolo
        response2 = client.post(
            "/protocolo",
            json={
                "sistema_origem": "VNA",
                "email_analista_responsavel": "vna@vna.com.br",
                "data_protocolo": "2025-03-12",
                "tipo": TipoArquivo.Video,
                "endereco_arquivo": "www.arquivo.com/exame.mp4",
            },
        )

        if verbose: print(response2.json())
        assert response2.status_code == 200
        assert response2.json()["sistema_origem"] == "VNA", "O nome do sistema de origem do protocolo deveria ser Cerner, Pacs, VNA"
        assert response2.json()["codigo"], "O protocolo deve ter um código"

        # fazendo o parse da resposta json para o objeto
        protocolo2 = Protocolo.model_validate(response2.json())
        assert str(protocolo2.codigo) == response2.json()["codigo"], "O código deve ser o mesmo"
        assert protocolo2.data_protocolo, "A data do protocolo deve ser informada"
        assert protocolo2.tipo, "O tipo do arquivo do protocolo deve ser informado"
        assert protocolo2.endereco_arquivo, "A localização do arquivo deve ser informado"


In [89]:
def recebeProtocolo(verbose = False) -> None:
    with TestClient(app) as client2:

        response3 = client2.get("/protocolo")
        assert response3.status_code == 200, "Código de resposta deve ser 200"
        if verbose: print(len(response3.json()))
        if verbose: print(response3.json()[0]["codigo"])
        # comentado após ter sido testado para testar a próxima validação
        # assert len(response.json()) == 1, "A Quantidade de protocolos está errada"

        # buscando protocolo por código
        response3 = client2.get(f"/protocolo/{response3.json()[0]['codigo']}")
        assert response3.status_code == 200
        # comentado após ter sido testado para testar a próxima validação
        # assert response3.json()["data_protocolo"] == "2025-03-12", "A data do protocolo é diferente da data informada"

        #verificando protocolo que não existe
        codigo_teste = uuid4()
        response4 = client2.get(f"/protocolo/{codigo_teste}")
        assert response4.status_code == 404
        assert response4.json()["mensagem"] == f"Protocolo de código {codigo_teste} não encontrado", (
            "Este protocolo não deve ser encontrado"
        )



In [90]:
# executando a simulação do fluxo de validação do sistema
if __name__ == "__main__":
    print(40*"")
    print("iniciando teste de envio de protocolo")
    enviaProtocolo(False)
    print("teste de envio de protocolo finalizado")
    print(40*"")


    print(40*"")
    print("iniciando teste de recebimento de protocolo")
    recebeProtocolo(False)
    print("teste de recebimento de protocolo finalizado")
    print(40*"")


iniciando teste de envio de protocolo
Recebendo protocolo
Recebendo protocolo
teste de envio de protocolo finalizado


iniciando teste de recebimento de protocolo
retornando lista de protocolos
realizando busca de protocolo por código
realizando busca de protocolo por código
teste de recebimento de protocolo finalizado

